In [1]:
from dataset import BPI2012Dataset
from parameters.dataset import BPI2012Parameters
from utils.save import get_json_dict
from parameters.training import TrainingParameters, LossParameters, OptimizerParameters
from parameters.model import PredNextBERTParameters
from model import PredNextBERT
from controller import TrainingController

In [2]:
bpi_2012_parms = BPI2012Parameters()

In [3]:
get_json_dict(bpi_2012_parms)

{'file_path': './data/event_logs/BPI_Challenge_2012.xes',
 'preprocessed_folder_path': './data/preprocessed/BPI_Challenge_2012_with_resource',
 'include_types': ['A', 'O', 'W'],
 'include_complete_only': True}

In [4]:
dataset = BPI2012Dataset(bpi_2012_parms)


| Preprocessed data loaded successfully: ./data/preprocessed/BPI_Challenge_2012_with_resource/AOW_CompleteOnly 


In [5]:
model_params = PredNextBERTParameters()

In [6]:
get_json_dict(model_params)

{'num_layers': 4,
 'model_dim': 32,
 'feed_forward_dim': 64,
 'num_heads': 4,
 'dropout_rate': 0.1}

In [7]:
model = PredNextBERT(
    activity_vocab = dataset.activity_vocab,
    parameters = model_params
)

In [8]:
train_params = TrainingParameters(
    stop_epoch=50,
)

In [9]:
get_json_dict(train_params)

{'stop_epoch': 1,
 'batch_size': 128,
 'verbose_freq': 250,
 'run_validation_freq': 80,
 'train_test_split_portion': [0.8, 0.1],
 'random_seed': 12345}

In [10]:
from parameters.enum import SelectableLoss, SelectableLrScheduler, SelectableOptimizer

In [11]:
loss_params = LossParameters(
    loss=SelectableLoss.CrossEntropy,
    from_logits=True
)

In [12]:
get_json_dict(loss_params)

{'loss': 'CrossEntropy', 'from_logits': True}

In [13]:
optim_params = OptimizerParameters(
    optimizer=SelectableOptimizer.Adam,
    lr_scheduler=SelectableLrScheduler.BERTScheduler,
)

In [14]:
get_json_dict(optim_params)

{'optimizer': 'Adam',
 'learning_rate': 0.005,
 'l2': 1e-10,
 'lr_scheduler': 'BERTScheduler',
 'lr_step_scheduler_step': 800,
 'lr_setp_scheduler_gamma': 0.8,
 'lr_exp_decay_scheduler_step': 1000,
 'lr_exp_decay_scheduler_rate': 0.96,
 'lr_exp_decay_scheduler_staircase': True,
 'SGD_momentum': 0.9}

In [15]:
controller = TrainingController(
    dataset,
    model,
    train_params,
    optim_params,
    loss_params,
)


| Running on /job:localhost/replica:0/task:0/device:CPU:0  


In [16]:
controller.model_info()

Model: "PredNextBERT"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
predict_next_encoder (Predic multiple                  35008     
_________________________________________________________________
dense_24 (Dense)             multiple                  858       
Total params: 35,866
Trainable params: 35,866
Non-trainable params: 0
_________________________________________________________________


In [17]:
test_accuracy = controller.train()


| Training records in logs/gradient_tape/20210704-053341 

| Total epochs: 1 

| Total steps: 82 

| Start epoch 0 

| Evaluation result | Loss [3.0305] | Accuracy [0.0503]  

| Performing Testing ... 

| Evaluation result | Loss [3.0148] | Accuracy [0.0482]  


In [19]:
save_folder_path = model.get_folder_path("bpi_2012_bert_training.ipynb", test_accuracy , dataset.info_str())

In [20]:
controller.save_parameters(save_folder_path)
model.save(save_folder_path)


| Parameters saved successfully to: ./SavedModels/0.0482_PredNextBERT_AOW_CompleteOnly_2021-07-04 05:48:56.605147  

| Vocabs saved successfully to: ./SavedModels/0.0482_PredNextBERT_AOW_CompleteOnly_2021-07-04 05:48:56.605147  

| Model saved successfully to: ./SavedModels/0.0482_PredNextBERT_AOW_CompleteOnly_2021-07-04 05:48:56.605147  


In [24]:
# controller.load_parameters(save_folder_path)
# model.load(save_folder_path)


| Model parameters loaded successfully from: ./SavedModels/0.0482_PredNextBERT_AOW_CompleteOnly_2021-07-04 05:48:56.605147  

| Vocab loaded successfully from: ./SavedModels/0.0482_PredNextBERT_AOW_CompleteOnly_2021-07-04 05:48:56.605147  

| Model loaded successfully from: ./SavedModels/0.0482_PredNextBERT_AOW_CompleteOnly_2021-07-04 05:48:56.605147  
